In [ ]:
from RoboticArm import *
from IK import *
import numpy as np
from simulation import Simulation, Controller
from model import MuJoCo_Model as Model

# ARM IK's model

In [ ]:
ik_model = viper300()

In [ ]:
# EE position at zero configuration (all angles are zeroed)
# Accoding to the IK model, the arm is pointing towards the x-axis, 
# y is upwards, and z is pointing at the chair
print(ik_model.get_xyz_numeric([0,0,0,0,0]).T[0])

In [ ]:
# Offset angles for the physical arm in relative to the IK mpdel
offset_relative_to_IK_Model = {1: 90, 2: 180, 3: 180, 4: 180, 
                               5: 180, 6: 0, 7: 180, 8: 0, 9: 0}

def robot_to_model_position(robot_position):

    return [     np.deg2rad(robot_position[1]-offset_relative_to_IK_Model[1]),
            -1 * np.deg2rad(robot_position[2]-offset_relative_to_IK_Model[2]),
            -1 * np.deg2rad(robot_position[4]-offset_relative_to_IK_Model[4]),
                 np.deg2rad(robot_position[6]-offset_relative_to_IK_Model[6]),
            -1 * np.deg2rad(robot_position[7]-offset_relative_to_IK_Model[7])]

def model_to_robot_position(model_position):
     
    f = [ np.rad2deg(     model_position[0])+offset_relative_to_IK_Model[1],
          np.rad2deg(-1 * model_position[1])+offset_relative_to_IK_Model[2],
          np.rad2deg(-1 * model_position[2])+offset_relative_to_IK_Model[4],
          np.rad2deg(     model_position[3])+offset_relative_to_IK_Model[6],
          np.rad2deg(-1 * model_position[4])+offset_relative_to_IK_Model[7]]

    return {1: int(f[0]), 2: int(f[1]), 3: int(f[1]), 4: int(f[2]),
            5: int(f[2]), 6: int(f[3]), 7: int(f[4]), 8: 180, 9: 180}


In [ ]:
# EE position in the physical arm's configuration space
print(Robot['Real']['Home'])

In [ ]:
# EE position in the model's configuration s pace
home_chair = robot_to_model_position(Robot['Real']['Home'])
print(home_chair)
# Going back to the physical arm's configuration space
print(model_to_robot_position(home_chair))

In [ ]:
# EE position in the model's task space
list(ik_model.get_xyz_numeric(home_chair).T[0])

### ARM physical model

In [ ]:
BASE_DIR = '/Users/elishai/Dropbox/1AFiles/NBEL G2/NBEL_repository/Adaptive_arm_control/'
  
model_name = 'NBEL'
model      = Model(BASE_DIR + 'arm_models/{}/{}.xml'.format(model_name, model_name))
controller = Controller(model)

In [ ]:
## EE position in the physical model's configuration space
p = [1, -1, -1, 1, -1] # y z z x z: accounting for direction of rotation
q_dic = {i: p[i]*v for i, v in enumerate (home_chair)}
print(q_dic)

In [ ]:
## EE Position in the physical model's task space
# Accoding to the physical model, the arm is poiting towards the y-axis, 
# z is upwards, and x is pointing at the chair
model.goto_null_position()                                  # Goto reference position
model.send_target_angles(q_dic)                             # Manipulate model
c = model.get_ee_position()                                 # Current position
model.visualize()
print(c)

In [ ]:
# Transforming EE position in the physical model's to the physical's arm task space
TRx = np.array([[1,  0, 0],
                [0,  0, 1],
                [0, -1, 0]])

TRz = np.array([[0,  1, 0],
                [-1,  0, 0],
                [0,  0, 1]])

TRxz = np.dot(TRx, TRz)
print(list(np.dot(TRxz, c)))

### Arm manipulation

In [ ]:
# EE position in the model's configuration s pace
home_chair = robot_to_model_position(Robot['Real']['Home'])
J_x = ik_model.calc_J_numeric(home_chair) # Calculate the jacobian
ux = [0.15, 0, 0.15]
u = np.dot(np.linalg.pinv(J_x), ux)
home_chair_m = home_chair + u

In [ ]:
print(ik_model.get_xyz_numeric(home_chair).T[0])
print(ik_model.get_xyz_numeric(home_chair_m).T[0])

In [ ]:
# Going back to the physical arm's configuration space
print(model_to_robot_position(home_chair_m))

### Arm configuration

### Arm Control

In [ ]:
arm = RoboticArm(Robot, COM_ID = '/dev/tty.usbserial-FT4NNWBP')

In [ ]:
arm.go_home()

In [ ]:
arm.set_position({1: 85, 2: 135+10, 3: 135+10, 4: 180+10, 5: 180+10, 6: 180, 7:135-20, 8:180+10, 9:180})

In [ ]:
arm.set_position({1: 85, 2: 135, 3: 135, 4: 180, 5: 180, 6: 180, 7:135, 8:180, 9:180})

In [ ]:
arm.destruct()